In [ ]:
import boto3
import json
from dotenv import load_dotenv

bucket_name = os.getenv('BUCKET_NAME_P')
lambda_function_name = os.getenv('LAMBDA_FUNCTION_NAME_P')

# Extensiones ignoradas
excluidos = ['.txt', '.csv', '.log']

s3 = boto3.client('s3')
lambda_client = boto3.client('lambda')

# Lista archivos en el bucket
response = s3.list_objects_v2(Bucket=bucket_name)
all_keys = [obj['Key'] for obj in response.get('Contents', [])]

# Filtra los archivos que NO terminan con extensiones excluidas
keys = [key for key in all_keys if not any(key.endswith(ext) for ext in excluidos)]

print(f"Se encontraron {len(keys)} archivos en el bucket '{bucket_name}'.")

# Invoca la Lambda por cada archivo
for key in keys:
    print(f"Invocando Lambda para el archivo: {key}")
    payload = json.dumps({'key': key})
    lambda_client.invoke(
        FunctionName=lambda_function_name,
        InvocationType='Event', 
        Payload=payload
    )

print("Todas las invocaciones han sido lanzadas.")